# Hugging Face Transformers

## 1. Library Imports

In [ ]:
import json
import numpy as np
import pandas as pd
import torch
import os
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorForTokenClassification,
    EarlyStoppingCallback
)
from datasets import Dataset as HFDataset
from datasets import DatasetDict
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## 3. Constant Definition 

In this cell, I´ll document the type of entities and their correspondant colors.

In [ ]:
# Define entity types and their descriptions
ENTITY_TYPES = {
    "ACTION": "Direct commands or actions mentioned in the message",
    "SITUATION": "Racing context or circumstance descriptions",
    "INCIDENT": "Accidents or on-track events",
    "STRATEGY_INSTRUCTION": "Strategic directives",
    "POSITION_CHANGE": "References to overtakes or positions",
    "PIT_CALL": "Specific calls for pit stops",
    "TRACK_CONDITION": "Mentions of the track's state",
    "TECHNICAL_ISSUE": "Mechanical or car-related problems",
    "WEATHER": "References to weather conditions"
}

# Color scheme for entity visualization
ENTITY_COLORS = {
    "ACTION": "#4e79a7",           # Blue
    "SITUATION": "#f28e2c",         # Orange
    "INCIDENT": "#e15759",          # Red
    "STRATEGY_INSTRUCTION": "#76b7b2", # Teal
    "POSITION_CHANGE": "#59a14f",   # Green
    "PIT_CALL": "#edc949",          # Yellow
    "TRACK_CONDITION": "#af7aa1",   # Purple
    "TECHNICAL_ISSUE": "#ff9da7",   # Pink
    "WEATHER": "#9c755f"            # Brown
}

print("Entity types defined:")
for entity, description in ENTITY_TYPES.items():
    print(f"  - {entity}: {description}")

Types of defined entities:
  - ACTION: Direct commands or actions mentioned in the message
  - SITUATION: Racing context or circumstance descriptions
  - INCIDENT: Accidents or on-track events
  - STRATEGY_INSTRUCTION: Strategic directives
  - POSITION_CHANGE: References to overtakes or positions
  - PIT_CALL: Specific calls for pit stops
  - TRACK_CONDITION: Mentions of the track's state
  - TECHNICAL_ISSUE: Mechanical or car-related problems
  - WEATHER: References to weather conditions


## 4. Load and Explore Data

In [11]:
# Load F1 radio data from JSON file
def load_f1_radio_data(json_file):
    """Load and explore F1 radio data from JSON file"""
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    print(f"Loaded {len(data)} messages from {json_file}")
    
    # Show sample structure
    if len(data) > 0:
        print("\nSample record structure:")
        sample = data[0]
        print(f"  Driver: {sample.get('driver', 'N/A')}")
        print(f"  Radio message: {sample.get('radio_message', 'N/A')[:100]}...")
        
        if 'annotations' in sample and len(sample['annotations']) > 1:
            if isinstance(sample['annotations'][1], dict) and 'entities' in sample['annotations'][1]:
                entities = sample['annotations'][1]['entities']
                print(f"  Number of entities: {len(entities)}")
                if len(entities) > 0:
                    entity = entities[0]
                    entity_text = sample['radio_message'][entity[0]:entity[1]]
                    print(f"  Sample entity: [{entity[0]}, {entity[1]}, '{entity_text}', '{entity[2]}']")
    
    return data



In [12]:
# Load the JSON data
json_file_path = "f1_radio_entity_annotations.json"
f1_data = load_f1_radio_data(json_file_path)

# Count entity types in the dataset
entity_counts = {}
for item in f1_data:
    if 'annotations' in item and len(item['annotations']) > 1:
        if isinstance(item['annotations'][1], dict) and 'entities' in item['annotations'][1]:
            for _, _, entity_type in item['annotations'][1]['entities']:
                entity_counts[entity_type] = entity_counts.get(entity_type, 0) + 1

print("\nEntity type distribution in dataset:")
for entity_type, count in sorted(entity_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  - {entity_type}: {count}")

Loaded 529 messages from f1_radio_entity_annotations.json

Sample record structure:
  Driver: 1
  Radio message: So don't forget Max, use your head please. Are we both doing it or what? You just follow my instruct...
  Number of entities: 3
  Sample entity: [82, 103, 'follow my instruction', 'ACTION']

Entity type distribution in dataset:
  - SITUATION: 255
  - ACTION: 165
  - STRATEGY_INSTRUCTION: 137
  - TECHNICAL_ISSUE: 137
  - WEATHER: 112
  - POSITION_CHANGE: 83
  - INCIDENT: 78
  - TRACK_CONDITION: 62
  - PIT_CALL: 42


## 5. Preprocessing F1 Radio Data

In [27]:
def preprocess_f1_data(data):
    """Extract and preprocess F1 radio data with valid annotations"""
    processed_data = []
    skipped_count = 0
    
    for item in data:
        if 'radio_message' not in item or 'annotations' not in item:
            skipped_count += 1
            continue
            
        text = item['radio_message']
        
        # Skip items with empty or null text
        if not text or text.strip() == "":
            skipped_count += 1
            continue
            
        # Extract entities if they exist in expected format
        if len(item['annotations']) > 1 and isinstance(item['annotations'][1], dict):
            annotations = item['annotations'][1]
            if 'entities' in annotations and annotations['entities']:
                entities = annotations['entities']
                
                # Add to processed data
                processed_data.append({
                    'text': text,
                    'entities': entities,
                    'driver': item.get('driver', None)
                })
            else:
                skipped_count += 1
        else:
            skipped_count += 1
    
    print(f"Processed {len(processed_data)} messages with valid annotations")
    print(f"Skipped {skipped_count} messages with missing or invalid annotations")
    
    # Show a sample of processed data
    if processed_data:
        sample = processed_data[10]
        print("\nSample processed message:")
        print(f"Text: {sample['text']}")
        print("Entities:")
        for start, end, entity_type in sample['entities']:
            entity_text = sample['text'][start:end]
            print(f"  - [{start}, {end}] '{entity_text}' ({entity_type})")
    
    return processed_data



In [26]:
# Preprocess the loaded data
processed_f1_data = preprocess_f1_data(f1_data)

Processed 399 messages with valid annotations
Skipped 130 messages with missing or invalid annotations

Sample processed message:
Text: I have to join the Slums because my tyres are f***ed. Understood Max. It might just be survival in that section of track, Max, if the rest of the track is dry.
Entities:
  - [87, 120] 'survival in that section of track' (ACTION)
  - [0, 24] 'I have to join the Slums' (STRATEGY_INSTRUCTION)
  - [127, 158] 'if the rest of the track is dry' (TRACK_CONDITION)
  - [127, 158] 'if the rest of the track is dry' (WEATHER)
